# Tracking Control Log Visualization 

In [ ]:
import os
from math import pi
from typing import cast

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.artist import Artist
from matplotlib.axes import Axes
from matplotlib.ticker import ScalarFormatter

In [ ]:
df = pd.read_csv(
    "tracking_control_node.csv",
    index_col=0,
    parse_dates=["timestamp"],
)

df.info()

In [ ]:
asc_flag = "x_vf" in df.columns

if asc_flag:
    front_label, rear_label = "vf", "vr"
    WHEEL_RADIUS = 0.875  # m
    DRIVER2WHEEL_RATIO = 23.5
else:
    front_label, rear_label = "f", "r"
    WHEEL_RADIUS = 0.85  # m
    DRIVER2WHEEL_RATIO = 24.67

RPM_TO_MPS = 1 / 60 * 2 * pi * WHEEL_RADIUS \
    / DRIVER2WHEEL_RATIO  # rpm^{-1} * m/s

In [ ]:
if asc_flag:
    TRANS_MAP_DIR_SUFFIX = "agv_ns_ros/data/AGV_Map/"
else:
    TRANS_MAP_DIR_SUFFIX = "agv_ns_ros_agv/data/AGV_Map/"

current_dir = os.path.dirname(os.getcwd())
while (parent_dir := os.path.dirname(current_dir)) != current_dir:
    current_dir = parent_dir
    map_dir_path = os.path.join(current_dir, TRANS_MAP_DIR_SUFFIX)
    if os.path.exists(map_dir_path):
        TRANS_MAP_DIR = map_dir_path
        break
else:
    raise FileNotFoundError("Failed to find the transponder map file!")

trans_map_candidates = [
    file_name
    for file_name in os.listdir(TRANS_MAP_DIR)
    if file_name.startswith("TransMap") and file_name.endswith(".csv")
]
if len(trans_map_candidates) < 1:
    raise FileNotFoundError("Trans map file not found!")
else:
    selected_trans_map_file_name = sorted(trans_map_candidates)[-1]
    print(f"Selected {selected_trans_map_file_name!r}.")

TRANS_MAP_PATH = os.path.join(TRANS_MAP_DIR, selected_trans_map_file_name)
df_trans_map = pd.read_csv(
    TRANS_MAP_PATH,
    names=["TransID", "AbsX", "AbsY", "LaneNB1", "LaneNB2"],
    index_col=0,
)
df_trans_map["x"] = df_trans_map["AbsX"] / 1000
df_trans_map["y"] = df_trans_map["AbsY"] / 1000
df_trans_map.info()

In [ ]:
def plot_transponders(ax: Axes) -> None | Artist:
    handle = None
    for id in df_trans_map.index:
        handle = ax.plot(
            cast(np.float64, df_trans_map.loc[id, "x"]),
            cast(np.float64, df_trans_map.loc[id, "y"]),
            "+",
            ms=5,
            color="purple",
            label="Transponders",
        )[0]
    return handle

## Log Timestamps

In [ ]:
min_timestamp, max_timestamp = df["timestamp"].min(), df["timestamp"].max()
timestamp_limit_padding = np.timedelta64(0, "s")
timestamp_limits = (
    min_timestamp - timestamp_limit_padding,
    max_timestamp + timestamp_limit_padding,
)
print(min_timestamp)
print(max_timestamp)

## Valid Flag

In [ ]:
df.plot(x="timestamp", y="valid", xlim=timestamp_limits)

## Brake State Before Filtering

In [ ]:
df.plot(x="timestamp", y="command_brake", xlim=timestamp_limits)

## Filter Out Invalid Data

In [ ]:
df = df[df["valid"] == 1]
# zone_offset = pd.Timedelta(8, "h")
# begin_time = pd.Timestamp(1728465234, unit="s") + zone_offset
# end_time = pd.Timestamp(1728466138, unit="s") + zone_offset
# print(begin_time)
# print(end_time)
# df = df[
#     # (df["timestamp"] >= begin_time)
#     # &
#     (df["timestamp"] <= end_time)
# ]
print("remained records", len(df))

## Localization (Vehicle Center)

In [ ]:
fig = plt.figure(figsize=(6, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

mask = (df["x_center"] != 0) & (df["y_center"] != 0)
correction_filter = (
    (df["x_center"].loc[mask] != df["x_estimate"].loc[mask])
    | (df["y_center"].loc[mask] != df["y_estimate"].loc[mask])
)
handle_real = ax.plot(
    df["x_center"].loc[mask],
    df["y_center"].loc[mask],
    "b.-",
    lw=1,
    ms=3,
    alpha=0.2,
    label="Real Path",
)[0]
handle_correction = ax.plot(
    df["x_center"].loc[correction_filter],
    df["y_center"].loc[correction_filter],
    "ro",
    ms=5,
    alpha=0.5,
    label="Correction Point(s)",
)[0]
handle_transponders = plot_transponders(ax)
handle_start = ax.plot(
    df["x_center"].loc[mask].iloc[0],
    df["y_center"].loc[mask].iloc[0],
    "y^",
    alpha=0.5,
    label="Start Point",
)[0]
ax.set(
    title="AGVLocalization",
    xlabel="x_center (m)",
    ylabel="y_center (m)",
    xlim=(df["x_center"].min() - 4, df["x_center"].max() + 4),
    ylim=(df["y_center"].min() - 4, df["y_center"].max() + 4),
    aspect="equal",
)
ax.xaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.legend(
    handles=[
        handle_real,
        handle_correction,
        handle_transponders,
        handle_start,
    ],
)
ax.grid()

pass

## Localization (Antenna Centers)

In [ ]:
fig = plt.figure(figsize=(6, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

path_handles: list[Artist]
path_kwargs = dict(ms=2, alpha=0.2)
if asc_flag:
    path_handles = [
        ax.plot(df["x_vf"], df["y_vf"], "r.-",
                **path_kwargs, label="front")[0],  # type: ignore
        ax.plot(df["x_vr"], df["y_vr"], "b.-",
                **path_kwargs, label="rear")[0],  # type: ignore
        ax.plot(df["x_f"], df["y_f"], "g.-",
                **path_kwargs, label="left")[0],  # type: ignore
        ax.plot(df["x_r"], df["y_r"], "c.-",
                **path_kwargs, label="right")[0],  # type: ignore
    ]
else:
    path_handles = [
        ax.plot(df["x_f"], df["y_f"], "r.-",
                **path_kwargs, label="front")[0],  # type: ignore
        ax.plot(df["x_r"], df["y_r"], "b.-",
                **path_kwargs, label="rear")[0],  # type: ignore
    ]

# start/stop direction
for i in (0, -1):
    if asc_flag:
        ax.plot(
            (df["x_vf"].dropna().iloc[i], df["x_vr"].dropna().iloc[i]),
            (df["y_vf"].dropna().iloc[i], df["y_vr"].dropna().iloc[i]),
            "--",
            color="gray",
            alpha=0.5,
        )
    ax.plot(
        (df["x_f"].dropna().iloc[i], df["x_r"].dropna().iloc[i]),
        (df["y_f"].dropna().iloc[i], df["y_r"].dropna().iloc[i]),
        "--",
        color="gray",
        alpha=0.5,
    )

# tranponders
handle_transponders = plot_transponders(ax)

# start points
handle_start = ax.plot(
    *zip(
        *[
            (
                df[f"x_{suffix}"].dropna().iloc[0],
                df[f"y_{suffix}"].dropna().iloc[0],
            )
            for suffix in (
                ("vf", "vr", "f", "r") if asc_flag else ("f", "r")
            )
        ]
    ),
    "ys",
    ms=8,
    alpha=0.5,
    label=f"start points",
)[0]

if asc_flag:
    x_max = np.nanmax(df[["x_f", "x_r", "x_vf", "x_vr"]].to_numpy())
    x_min = np.nanmin(df[["x_f", "x_r", "x_vf", "x_vr"]].to_numpy())
    y_max = np.nanmax(df[["y_f", "y_r", "y_vf", "y_vr"]].to_numpy())
    y_min = np.nanmin(df[["y_f", "y_r", "y_vf", "y_vr"]].to_numpy())
else:
    x_max = np.nanmax(df[["x_f", "x_r"]].to_numpy())
    x_min = np.nanmin(df[["x_f", "x_r"]].to_numpy())
    y_max = np.nanmax(df[["y_f", "y_r"]].to_numpy())
    y_min = np.nanmin(df[["y_f", "y_r"]].to_numpy())
x_mid = (x_max + x_min) / 2
y_mid = (y_max + y_min) / 2
view_radius = max(x_max - x_min, y_max - y_min) / 2 + 4

ax.set(
    title="AGVLocalization - Antenna Center Paths",
    xlabel="x (m)",
    ylabel="y (m)",
    xlim=(np.floor(x_mid - view_radius), np.ceil(x_mid + view_radius)),
    ylim=(np.floor(y_mid - view_radius), np.ceil(y_mid + view_radius)),
)
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.set_aspect("equal")
ax.legend(handles=[*path_handles, handle_transponders, handle_start])
ax.grid()

pass

## Localization (TransInAgv)

In [ ]:
if "trans_in_agv_fx" in df:

    fig = plt.figure(figsize=(12, 6), dpi=150)
    fig.set_facecolor("#fff")
    ax_x = fig.add_subplot()
    ax_y = ax_x.twinx()

    handles = []

    for label, color in (
        ("trans_in_agv_fx", "red"),
        ("trans_in_agv_fy", "orange"),
        ("trans_in_agv_rx", "green"),
        ("trans_in_agv_ry", "blue"),
    ):
        ax = ax_x if label[-1] == "x" else ax_y
        handle = ax.plot(
            df["timestamp"],
            df[label],
            "o",
            ms=3,
            color=color,
            alpha=0.3,
            label=label,
        )[0]
        handles.append(handle)

    ax_x.set(
        title="trans_in_agv_*",
        xlabel="Timestamp",
        ylabel="X (m)",
        xlim=timestamp_limits,
    )
    ax_y.set_ylabel("Y (m)")
    ax_x.legend(handles=handles)
    ax_x.grid()

pass

## Vehicle Center

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()
ax_twin = ax.twinx()

line_1 = ax.plot(
    df["timestamp"],
    df["x_center"],
    "b.-",
    ms=2,
    alpha=0.5,
    label="center x",
)[0]
line_2 = ax_twin.plot(
    df["timestamp"],
    df["y_center"],
    "r.-",
    ms=2,
    alpha=0.5,
    label="center y",
)[0]
line_3 = ax.plot(
    df["timestamp"],
    df["x_estimate"],
    "c.--",
    ms=2,
    alpha=0.3,
    label="x_estimate",
)[0]
line_4 = ax_twin.plot(
    df["timestamp"],
    df["y_estimate"],
    "y.--",
    ms=2,
    alpha=0.3,
    label="y_estimate",
)[0]
ax.set(
    title="Vehicle Center Position",
    xlabel="Timestamp",
    ylabel="x (m)",
    xlim=timestamp_limits,
    ylim=(df["x_center"].min() - 2, df["x_center"].max() + 2),
)
ax_twin.set(
    ylabel="y (m)",
    ylim=(df["y_center"].min() - 2, df["y_center"].max() + 2),
)
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax_twin.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.legend(handles=(line_1, line_2, line_3, line_4))
ax.grid()

print(
    "Travelled X:",
    df["x_center"].dropna().iloc[-1] -
    df["x_center"].dropna().iloc[0]
)
print(
    "Travelled Y:",
    df["y_center"].dropna().iloc[-1] -
    df["y_center"].dropna().iloc[0]
)

pass

## Front Antenna Center

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()
ax_twin = ax.twinx()

line_1 = ax.plot(
    df["timestamp"],
    df["x_" + front_label],
    "b.-",
    ms=2,
    alpha=0.5,
    label="front x",
)[0]
line_2 = ax_twin.plot(
    df["timestamp"],
    df["y_" + front_label],
    "r.-",
    ms=2,
    alpha=0.5,
    label="front y",
)[0]
line_3 = ax.plot(
    df["timestamp"],
    df["x_front_feedforward"],
    "c.--",
    ms=2,
    alpha=0.3,
    label="x_front_feedforward",
)[0]
line_4 = ax_twin.plot(
    df["timestamp"],
    df["y_front_feedforward"],
    "y.--",
    ms=2,
    alpha=0.3,
    label="y_front_feedforward",
)[0]
ax.set(
    title="Front Antenna Position",
    xlabel="Time",
    ylabel="x (m)",
    xlim=timestamp_limits,
    ylim=(df["x_" + front_label].min() - 2, df["x_" + front_label].max() + 2),
)
ax_twin.set(
    ylabel="y (m)",
    ylim=(df["y_" + front_label].min() - 2, df["y_" + front_label].max() + 2),
)
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax_twin.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.legend(handles=(line_1, line_2, line_3, line_4))
ax.grid()

print(
    "Travelled X:",
    df["x_" + front_label].dropna().iloc[-1] -
    df["x_" + front_label].dropna().iloc[0]
)
print(
    "Travelled Y:",
    df["y_" + front_label].dropna().iloc[-1] -
    df["y_" + front_label].dropna().iloc[0]
)

pass

## Rear Antenna Center

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()
ax_twin = ax.twinx()

line_1 = ax.plot(
    df["timestamp"],
    df["x_" + rear_label],
    "b.-",
    ms=2,
    alpha=0.5,
    label="rear x",
)[0]
line_2 = ax_twin.plot(
    df["timestamp"],
    df["y_" + rear_label],
    "r.-",
    ms=2,
    alpha=0.5,
    label="rear y",
)[0]
line_3 = ax.plot(
    df["timestamp"],
    df["x_rear_feedforward"],
    "c.--",
    ms=2,
    alpha=0.3,
    label="x_rear_feedforward",
)[0]
line_4 = ax_twin.plot(
    df["timestamp"],
    df["y_rear_feedforward"],
    "y.--",
    ms=2,
    alpha=0.3,
    label="y_rear_feedforward",
)[0]
ax.set(
    title="rear Antenna Position",
    xlabel="Time",
    ylabel="x (m)",
    xlim=timestamp_limits,
    ylim=(df["x_" + rear_label].min() - 2, df["x_" + rear_label].max() + 2),
)
ax_twin.set(
    ylabel="y (m)",
    ylim=(df["y_" + rear_label].min() - 2, df["y_" + rear_label].max() + 2),
)
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax_twin.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.legend(handles=(line_1, line_2, line_3, line_4))
ax.grid()

print(
    "Travelled X:",
    df["x_" + rear_label].dropna().iloc[-1] -
    df["x_" + rear_label].dropna().iloc[0]
)
print(
    "Travelled Y:",
    df["y_" + rear_label].dropna().iloc[-1] -
    df["y_" + rear_label].dropna().iloc[0]
)

pass

## Antenna Center Feedforward

In [ ]:
fig = plt.figure(figsize=(6, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(df["x_front_feedforward"], df["y_front_feedforward"],
        "r.-", ms=2, alpha=0.2, label="front")
ax.plot(df["x_rear_feedforward"], df["y_rear_feedforward"],
        "b.-", ms=2, alpha=0.2, label="rear")

x_max = np.nanmax(df[["x_front_feedforward", "x_rear_feedforward"]].to_numpy())
x_min = np.nanmin(df[["x_front_feedforward", "x_rear_feedforward"]].to_numpy())
y_max = np.nanmax(df[["y_front_feedforward", "y_rear_feedforward"]].to_numpy())
y_min = np.nanmin(df[["y_front_feedforward", "y_rear_feedforward"]].to_numpy())
x_mid = (x_max + x_min) / 2
y_mid = (y_max + y_min) / 2
view_radius = max(x_max - x_min, y_max - y_min) / 2 + 1

ax.set(
    title="AGVLocalization - Feedforward Paths",
    xlabel="x (m)",
    ylabel="y (m)",
    xlim=(np.floor(x_mid - view_radius), np.ceil(x_mid + view_radius)),
    ylim=(np.floor(y_mid - view_radius), np.ceil(y_mid + view_radius)),
)
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
ax.set_aspect("equal")
ax.legend()
ax.grid()

pass

## Vehicle Heading

### Final Heading

In [ ]:
fig = plt.figure(figsize=(8, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(
    df["timestamp"],
    df["heading_estimate"] / pi * 180,
    "r.-",
    alpha=0.5,
    label="estimated heading",
)
ax.plot(
    df["timestamp"],
    df["heading"] / pi * 180,
    "b.-",
    alpha=0.5,
    label="resultant heading",
)
for y in (-180, -90, 0, 90, 180):
    ax.axhline(y, ls="--", color="gray", alpha=0.8)
ax.set(
    title="heading",
    xlabel="Time",
    ylabel="Heading (deg)",
    xlim=timestamp_limits,
)
ax.legend()
ax.grid()

pass

### Heading Computed from One Antenna

In [ ]:
if "heading_one_antenna" in df.columns:

    fig = plt.figure(figsize=(8, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()

    mask = df["heading_one_antenna"] != 0
    ax.plot(
        df["timestamp"][mask],
        df["heading_one_antenna"][mask] / pi * 180,
        "b.",
        alpha=0.5,
    )
    ax.set(
        title="heading_one_antenna",
        xlabel="Time",
        ylabel="Heading (deg)",
        xlim=timestamp_limits,
    )
    ax.grid()

    pass

### Heading Computed from Two Antennas

In [ ]:
fig = plt.figure(figsize=(8, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

mask = df["heading_two_antennas"] != 0
ax.plot(
    df["timestamp"][mask],
    df["heading_two_antennas"][mask] / pi * 180,
    "b.",
    alpha=0.5,
)
ax.set(
    title="heading_two_antennas",
    xlabel="Time",
    ylabel="Heading (deg)",
    xlim=timestamp_limits,
)
ax.grid()

pass

## Running State

In [ ]:
fig = plt.figure(figsize=(8, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(df["timestamp"], df["running_state"], "b.-", alpha=0.5)
ax.set(
    title="RunningState",
    xlabel="Time",
    ylabel="Value",
    xlim=timestamp_limits,
    yticks=list(range(-1, 7 + 1)),
    yticklabels=(
        "MOTION_UNCERTAIN",
        "MOTION_STOPPING",
        "MOTION_STOP",
        "MOTION_STARTING",
        "MOTION_STRAIGHT",
        "MOTION_QTURN",
        "MOTION_CRABWISE",
        "MOTION_UTURN",
        "MOTION_STURN",
    ),
)
ax.grid()

pass

## Path Indices

### Index Flags

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("front_index_flag", "red"),
    ("rear_index_flag", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        color=color,
        ms=2,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="*_index_flag",
    xlabel="Time",
    ylabel="Flag",
    xlim=timestamp_limits,
    yticks=range(0, 1 + 1),
    ylim=(-0.5, 1.5),
)
ax.legend()
ax.grid()

pass

### Front Path Indices

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("front_index_angle", "red"),
    ("front_index_distance", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        color=color,
        ms=2,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="front_index_*",
    xlabel="Time",
    ylabel="Index",
    xlim=timestamp_limits,
    yticks=range(-1, 5),
    ylim=(-1.5, 4.5),
)
ax.legend()
ax.grid()

pass

### Rear Path Indices

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("rear_index_angle", "red"),
    ("rear_index_distance", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        color=color,
        ms=2,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="rear_index_*",
    xlabel="Time",
    ylabel="Index",
    xlim=timestamp_limits,
    yticks=range(-1, 5),
    ylim=(-1.5, 4.5),
)
ax.legend()
ax.grid()

pass

## Provided Values

### Rel X/Y

In [ ]:
fig = plt.figure(figsize=(12, 6), dpi=150)
fig.set_facecolor("#fff")
ax_x = fig.add_subplot()
ax_y = ax_x.twinx()

handles = []

for label, color in (
    ("rel_x_f", "red"),
    ("rel_y_f", "orange"),
    ("rel_x_r", "green"),
    ("rel_y_r", "blue"),
):
    mask = (df["new_valid_data_arrived_" + label[-1]] == 1) & (
        df["trans_data_get_" + label[-1]] == 1
    )
    ax = ax_x if label[4] == "x" else ax_y
    handle = ax.plot(
        df["timestamp"].loc[mask],
        df[label].loc[mask] / 1000,
        "o",
        ms=3,
        color=color,
        alpha=0.3,
        label=label,
    )[0]
    handles.append(handle)

ax_x.set(
    title="rel_x/y",
    xlabel="Time",
    ylabel="Relative X (m)",
    xlim=timestamp_limits,
)
ax_y.set_ylabel("Relative Y (m)")
ax_x.legend(handles=handles)
ax_x.grid()

pass

### Omega

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(df["timestamp"], df["omega_z"], "b.-", ms=2, alpha=0.5)
ax.set(
    title="omega_z",
    xlabel="Time",
    ylabel="Omega Z",
    xlim=timestamp_limits,
)
ax.grid()

pass

### Provided Steer Angles

In [ ]:
fig = plt.figure(figsize=(12, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

y_min = np.inf
y_max = -np.inf

for label, color in (
    ("steer_degree_fl", "red"),
    ("steer_degree_fr", "orange"),
    ("steer_degree_rl", "green"),
    ("steer_degree_rr", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        ms=1,
        color=color,
        alpha=0.3,
        label=label,
    )
    v_min = df[label].min()
    v_max = df[label].max()
    if v_min < y_min:
        y_min = v_min
    if v_max > y_max:
        y_max = v_max

Y_STEP = 1
y_min = (y_min // Y_STEP) * Y_STEP
y_max = (y_max // Y_STEP) * Y_STEP
y_limits = (
    y_min - Y_STEP,
    y_max + Y_STEP,
)

ax.set(
    title="steer_degree_*",
    xlabel="Time",
    ylabel="Provided Steer Angle (deg)",
    xlim=timestamp_limits,
    # yticks=np.arange(
    #     y_limits[0],
    #     y_limits[1] + Y_STEP,
    #     Y_STEP,
    # ),
    ylim=y_limits,
)
ax.legend(loc="center left")
ax.grid()

pass

### Provided Syn. Steer Degree

In [ ]:
fig = plt.figure(figsize=(12, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("steer_angle_fs", "red"),
    ("steer_angle_rs", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label] * 180 / pi,
        ".-",
        color=color,
        ms=2,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="steer_angle_*",
    xlabel="Time",
    ylabel="Provided Steer Angle (deg)",
    xlim=timestamp_limits,
    ylim=(-45, 45),
    yticks=np.arange(-45, 45 + 1, 5),
)
ax.legend()
ax.grid()

pass

### Provided Velocity

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("v_fl", "red"),
    ("v_fr", "orange"),
    ("v_rl", "green"),
    ("v_rr", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        ms=2,
        color=color,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="v_*",
    xlabel="Time",
    ylabel="Provided Velocity (m/s)",
    xlim=timestamp_limits,
)
ax.legend(loc="upper left")
ax.grid()

pass

### Provided Syn. Velocity

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("v_fs", "red"),
    ("v_rs", "blue"),
    ("v_s", "pink"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        ms=2,
        color=color,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="v_*",
    xlabel="Time",
    ylabel="Provided Velocity (m/s)",
    xlim=timestamp_limits,
)
ax.legend(loc="upper left")
ax.grid()

pass

## Global Velocity

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("v_gx", "red"),
    ("v_gy", "blue"),
    ("v_gs", "green"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        ms=2,
        color=color,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="v_g*",
    xlabel="Time",
    ylabel="Global Velocity (m/s)",
    xlim=timestamp_limits,
)
ax.legend()
ax.grid()

pass

## Computed RunDirection

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(df["timestamp"], df["run_direction"], "b.-", ms=2, alpha=0.5)
ax.set(
    title="MotionControlData.RunDirection (Computed)",
    xlabel="Time",
    ylabel="run_direction",
    xlim=timestamp_limits,
    yticks=(-1, 0, 1),
    ylim=(-1.5, 1.5),
)
ax.grid()

pass

## Computed Offset

### Computed Offset Y

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.axhline(0.03, ls="--", color="orange", alpha=0.5)
ax.axhline(-0.03, ls="--", color="orange", alpha=0.5)

for label, color in (
    ("offset_y_front", "red"),
    ("offset_y_rear", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        color=color,
        ms=2,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="offset_y_*",
    xlabel="Time",
    ylabel="Offset Y (m)",
    xlim=timestamp_limits,
)
ax.legend()
ax.grid()

pass

### Computed Offset X

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("offset_x_front", "red"),
    ("offset_x_rear", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        color=color,
        ms=2,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="offset_x_*",
    xlabel="Time",
    ylabel="Offset X (m)",
    xlim=timestamp_limits,
)
ax.legend()
ax.grid()

pass

## Diff Angle to Target

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("diff_angle_to_target_front", "red"),
    ("diff_angle_to_target_rear", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label] * 180 / pi,
        ".-",
        color=color,
        ms=2,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="diff_angle_to_target_*",
    xlabel="Time",
    ylabel="Angle (deg)",
    xlim=timestamp_limits,
)
ax.legend()
ax.grid()

pass

## PID Parameters

### $K_{\text{p}}$

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("kp_f", "red"),
    ("kp_r", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        ms=2,
        color=color,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="kp_*",
    xlabel="Time",
    ylabel=R"$K_{\text{p}}$",
    xlim=timestamp_limits,
)
ax.legend()
ax.grid()

pass

### $K_{\text{i}}$

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("ki_f", "red"),
    ("ki_r", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        ms=2,
        color=color,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="ki_*",
    xlabel="Time",
    ylabel=R"$K_{\text{i}}$",
    xlim=timestamp_limits,
)
ax.legend()
ax.grid()

pass

### $K_{\text{d}}$

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("kd_f", "red"),
    ("kd_r", "blue"),
):
    ax.plot(
        df["timestamp"],
        df[label],
        ".-",
        ms=2,
        color=color,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="kd_*",
    xlabel="Time",
    ylabel=R"$K_{\text{d}}$",
    xlim=timestamp_limits,
)
ax.legend()
ax.grid()

pass

## PID Output (Cross Component)

In [ ]:
if "vy_cross_f" in df:

    fig = plt.figure(figsize=(12, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()
    ax_twin = ax.twinx()

    for label, color, twin in (
        ("vy_cross_f_raw", "red", True),
        ("vy_cross_f", "orange", False),
        ("vy_cross_r_raw", "blue", True),
        ("vy_cross_r", "green", False),
    ):
        (ax_twin if twin else ax).plot(
            df["timestamp"],
            df[label],
            ".-",
            color=color,
            ms=2,
            alpha=0.3,
            label=label,
        )

    ax.set(
        title="vy_cross_* -- Cross Component Output from PID (before/after ValueLimit)",
        xlabel="Time",
        ylabel="After ValueLimit",
        xlim=timestamp_limits,
        # ylim=(-0.1, 0.1),
    )
    ax.legend(loc="center left")
    ax.grid(axis="y")

    ax_twin.set(
        ylabel="Before ValueLimit",
    )
    ax_twin.legend(loc="center right")

    pass

## PID Output (beta)

In [ ]:
if "beta" in df:

    fig = plt.figure(figsize=(12, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()

    ax.plot(
        df["timestamp"],
        df["beta"] * 180 / pi,
        ".-",
        color="#19f",
        alpha=0.5,
    )

    ax.set(
        title="beta -- Angle Output from PID",
        xlabel="Time",
        ylabel="Angle (deg)",
        xlim=timestamp_limits,
    )
    ax.grid(axis="y")

    pass

## PID Output (SteerAngle_Feedback)

In [ ]:
if "steer_angle_feedback_f" in df:

    fig = plt.figure(figsize=(12, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()

    for label, color in (
        ("steer_angle_feedback_f", "red"),
        ("steer_angle_feedback_r", "blue"),
    ):
        ax.plot(
            df["timestamp"],
            df[label] * 180 / pi,
            ".-",
            color=color,
            ms=2,
            alpha=0.3,
            label=label,
        )

    ax.set(
        title="steer_angle_feedback_* -- Output of New PID",
        xlabel="Time",
        ylabel="Angle (deg)",
        xlim=timestamp_limits,
        # ylim=(-8, 8),
    )
    ax.legend()
    ax.grid(axis="y")

    pass

## PID Output (SteerAngle)

In [ ]:
if "pid_steer_angle_f" in df:

    fig = plt.figure(figsize=(12, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()

    for label, color in (
        ("pid_steer_angle_f", "red"),
        ("pid_steer_angle_r", "blue"),
    ):
        ax.plot(
            df["timestamp"],
            df[label] * 180 / pi,
            ".-",
            color=color,
            ms=2,
            alpha=0.3,
            label=label,
        )

    ax.set(
        title="steer_angle_* -- Output of New PID (Feedback Plus Feedforward)",
        xlabel="Time",
        ylabel="Angle (deg)",
        xlim=timestamp_limits,
        # ylim=(-8, 8),
    )
    ax.legend()
    ax.grid(axis="y")

    pass

## Arrival Flag

In [ ]:
fig = plt.figure(figsize=(12, 3), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(df["timestamp"], df["target_pos_arrived"], "b.-", alpha=0.5)
ax.set(
    title="target_pos_arrived",
    xlabel="Time",
    xlim=timestamp_limits,
    yticks=(0, 1),
    yticklabels=("False", "True"),
    ylim=(-0.5, 1.5),
)
ax.grid()

pass

### Front Position After Arrival Flag

In [ ]:
if (df["target_pos_arrived"] != 1).all():
    first_arrival_index = -1
    print("not arrived")
else:

    first_arrival_index = df[df["target_pos_arrived"] == 1].index[0]
    print(first_arrival_index)
    print(df.loc[first_arrival_index, "timestamp"])
    print(df.loc[first_arrival_index, "x_" + front_label])
    print(df.loc[first_arrival_index, "y_" + front_label])

    fig = plt.figure(figsize=(10, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()
    ax_twin = ax.twinx()

    line_1 = ax.plot(
        df["timestamp"].loc[first_arrival_index:],
        df["x_" + front_label].loc[first_arrival_index:],
        "b.-",
        ms=2,
        alpha=0.5,
        label="front x",
    )[0]
    line_2 = ax_twin.plot(
        df["timestamp"].loc[first_arrival_index:],
        df["y_" + front_label].loc[first_arrival_index:],
        "r.-",
        ms=2,
        alpha=0.5,
        label="front y",
    )[0]

    ax.set(
        title="Front Antenna Position After Arrival Detected",
        xlabel="Time",
        ylabel="x (m)",
    )
    ax_twin.set_ylabel("y (m)")
    ax.legend(handles=(line_1, line_2), loc="center right")
    ax.grid()

    ax_twin.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
    ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))

    pass

### Front Speed After Arrival Flag

In [ ]:
if (df["target_pos_arrived"] != 1).all():
    first_arrival_index = -1
    print("not arrived")
else:

    first_arrival_index = df[df["target_pos_arrived"] == 1].index[0]
    print(first_arrival_index)
    print(df.loc[first_arrival_index, "timestamp"])
    print(
        df.loc[first_arrival_index, "command_speed_fs_rpm"]
        * RPM_TO_MPS  # type: ignore
    )

    fig = plt.figure(figsize=(10, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()

    line_1 = ax.plot(
        df["timestamp"].loc[first_arrival_index:],
        (
            df["command_speed_fs_rpm"].loc[first_arrival_index:] * RPM_TO_MPS
        ),
        "b.-",
        ms=2,
        alpha=0.5,
        label="command_speed_fs_rpm",
    )[0]
    line_2 = ax.plot(
        df["timestamp"].loc[first_arrival_index:],
        (
            df["command_speed_rs_rpm"].loc[first_arrival_index:] * RPM_TO_MPS
        ),
        "r.-",
        ms=2,
        alpha=0.5,
        label="command_speed_rs_rpm",
    )[0]

    ax.set(
        title="Front Antenna Speed After Arrival Detected",
        xlabel="Time",
        ylabel="Speed (m/s)",
    )
    ax.legend(handles=(line_1, line_2), loc="center right")
    ax.grid()

    ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))

    pass

## Set Points

### Command Angles

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("command_steer_angle_fl", "orange"),
    ("command_steer_angle_fr", "yellow"),
    ("command_steer_angle_rl", "green"),
    ("command_steer_angle_rr", "blue"),
    ("command_steer_angle_fs", "pink"),
    ("command_steer_angle_rs", "cyan"),
):
    v = df[label] / 10
    # v = v[1600:1800]
    print(f"{label} ∈ [{v.min():.1f}, {v.max():.1f}]")
    ax.plot(
        df["timestamp"],
        v,
        ".-",
        ms=2,
        color=color,
        alpha=0.3,
        label=label,
    )

ax.set(
    title="command_steer_angle_*",
    xlabel="Time",
    ylabel="Steer Angle (deg)",
    xlim=timestamp_limits,
)
ax.legend()
ax.grid()

pass

### Command Speed

In [ ]:
fig = plt.figure(figsize=(12, 6), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

for label, color in (
    ("command_speed_fl_rpm", "red"),
    ("command_speed_fr_rpm", "orange"),
    ("command_speed_rl_rpm", "green"),
    ("command_speed_rr_rpm", "blue"),
    ("command_speed_fs_rpm", "pink"),
    ("command_speed_rs_rpm", "cyan"),
):
    ax.plot(
        df["timestamp"],
        df[label] * RPM_TO_MPS,
        "o-",
        color=color,
        alpha=0.2,
        label=label,
        ms=2,
    )

ax.set(
    title="command_speed_*",
    xlabel="Time",
    ylabel="Command Speed (m/s)",
    xlim=timestamp_limits,
    # ylim=(-1, 1),
)
ax.legend()
ax.grid()

pass

### Brake Command

In [ ]:
fig = plt.figure(figsize=(12, 4), dpi=150)
fig.set_facecolor("#fff")
ax = fig.add_subplot()

ax.plot(df["timestamp"], df["command_brake"], "b.-", alpha=0.5)
ax.set(
    title="command_brake",
    xlabel="Time",
    xlim=timestamp_limits,
    ylim=(-0.5, 1.5),
    yticks=(0, 1),
    yticklabels=("BRAKE_ON", "BRAKE_OFF"),
)
ax.grid()

pass

### Brake Point

In [ ]:
if first_arrival_index >= 0:
    first_brake_index = (
        df.iloc[(len(df) // 2):].loc[df["command_brake"] == 0].index[0]
    )
    print(first_brake_index)
    print(df.loc[first_brake_index, "timestamp"])
    print(df.loc[first_brake_index, "x_" + front_label])
    print(df.loc[first_brake_index, "y_" + front_label])

#### Front Antenna Position After Brake Command

In [ ]:
if first_arrival_index >= 0:

    fig = plt.figure(figsize=(10, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()
    ax_twin = ax.twinx()

    line_1 = ax.plot(
        df["timestamp"].loc[first_brake_index:],
        df["x_" + front_label].loc[first_brake_index:],
        "b.-",
        ms=2,
        alpha=0.5,
        label="front x",
    )[0]
    line_2 = ax_twin.plot(
        df["timestamp"].loc[first_brake_index:],
        df["y_" + front_label].loc[first_brake_index:],
        "r.-",
        ms=2,
        alpha=0.5,
        label="front y",
    )[0]
    ax.set(
        title="Front Antenna Position After Brake Point",
        xlabel="Time",
        ylabel="x (m)",
    )
    ax_twin.set_ylabel("y (m)")
    ax.legend(handles=(line_1, line_2), loc="upper center")
    ax.grid()

    ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))
    ax_twin.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))

    pass

#### Front Speed After Brake Command

In [ ]:
if first_arrival_index >= 0:

    fig = plt.figure(figsize=(12, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()

    for label, color in (
        ("command_speed_fl_rpm", "red"),
        ("command_speed_fr_rpm", "orange"),
        ("v_fl", "green"),
        ("v_fr", "blue"),
    ):
        s_speed = df[label].loc[first_brake_index:]
        if label.endswith("_rpm"):
            s_speed = s_speed * RPM_TO_MPS
        ax.plot(
            df["timestamp"].loc[first_brake_index:],
            s_speed,
            ".-",
            color=color,
            alpha=0.3,
            label=label,
        )

    ax.set(
        title="Speed After Brake Point",
        xlabel="Time",
        ylabel="Speed (m/s)",
    )
    ax.legend()
    ax.grid()

    pass

#### Rel X/Y After Brake Command

In [ ]:
if first_arrival_index >= 0:

    fig = plt.figure(figsize=(10, 4), dpi=150)
    fig.set_facecolor("#fff")
    ax = fig.add_subplot()
    ax_twin = ax.twinx()

    line_1 = ax.plot(
        df["timestamp"].loc[first_brake_index:],
        df["rel_x_f"].loc[first_brake_index:],
        "r.-",
        ms=2,
        alpha=0.5,
        label="rel_x_f",
    )[0]
    line_2 = ax_twin.plot(
        df["timestamp"].loc[first_brake_index:],
        df["rel_y_f"].loc[first_brake_index:],
        "b.-",
        ms=2,
        alpha=0.5,
        label="rel_y_f",
    )[0]
    line_3 = ax.plot(
        df["timestamp"].loc[first_brake_index:],
        df["rel_x_r"].loc[first_brake_index:],
        "g.-",
        ms=2,
        alpha=0.5,
        label="rel_x_r",
    )[0]
    line_4 = ax_twin.plot(
        df["timestamp"].loc[first_brake_index:],
        df["rel_y_r"].loc[first_brake_index:],
        "c.-",
        ms=2,
        alpha=0.5,
        label="rel_y_r",
    )[0]

    ax.set(
        title="Rel X/Y After Brake Point",
        xlabel="Time",
        ylabel="x (m)",
    )
    ax_twin.set_ylabel("y (m)")
    ax.legend(handles=(line_1, line_2, line_3, line_4))
    ax.grid()

    ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False))

    pass